# Demo notebook for downloading TG211 data from MSK PIXNAT for analysis

In [2]:
!pip install pyxnat

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyxnat-1.4.tar.gz (78 kB)
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.9 MB 4.7 MB/s            ��███████▍           | 4.4 MB 4.7 MB/s eta 0:00:01
  Using cached pathlib-1.0.1.tar.gz (49 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for pyxnat: filename=pyxnat-1.4-py3-none-any.whl size=92687 sha256=114bf752464dfb595653433c81affc84d7d7d2f03d24db2f7d4a38bb695cde46
  Stored in directory: /home/locastre/.cache/pip/wheels/ab/8c/88/43b445801349ef91dea1a2f7e572f20b9940320e27fb985faa
  Created wheel for pathlib: filename=pathlib-1.0.1-py3-none-any.whl size=14348 sha256=18c9ddc8f3c9ae8b5b16aa588ca0a8eb5d0e05edf196cab8e3e2f68054e2dc7c
  Stored in directory: /home/locastre/.cache/pip/wheels/59/02/2f/ff4a3e16a518feb111ae1405908094483ef56fec0dfa39e571
Successfully built pyxnat pathlib


In [3]:
from pyxnat import Interface
import os
from glob import glob

## Connect to host

In [4]:
host = 'http://pixnat.mskcc.org'

In [6]:
xnat = Interface(host)

User:  locastre
 ·············


In [7]:
project_id = 'TG211_PAS'

In [8]:
xproj = xnat.select.project(project_id)

In [9]:
MILPPAB_list = []
UCLPTLU_list = []

In [10]:
xexplist = xproj.experiments('*').get()

### Get all UCL scans

In [11]:
for exp_id in xexplist:
    xexp = xnat.select.project(project_id).experiment(exp_id)
    label_id = xexp.attrs.get('label')
    if 'MILPPAB' in label_id:
        MILPPAB_list.append(exp_id)
    elif 'UCLPTLU' in label_id:
        UCLPTLU_list.append(exp_id)

In [12]:
UCLPTLU_list

['PI_XNAT_E04867',
 'PI_XNAT_E04868',
 'PI_XNAT_E04869',
 'PI_XNAT_E04870',
 'PI_XNAT_E04871',
 'PI_XNAT_E04872',
 'PI_XNAT_E04873',
 'PI_XNAT_E04874',
 'PI_XNAT_E04875',
 'PI_XNAT_E04876']

## Demo download of data to local folder

In [13]:
download_dir = 'demo'

In [14]:
os.makedirs(download_dir,exist_ok=True)

### Choose first UCL exam in list

In [15]:
exp_id = UCLPTLU_list[0]
print(exp_id)

PI_XNAT_E04867


In [16]:
xexp = xnat.select.project(project_id).experiment(exp_id)
label_id = xexp.attrs.get('label')
print(label_id)

UCLPTLU_02


### Get planC from first exam

In [16]:
cerrdownloadpath = os.path.join(download_dir,label_id)
os.makedirs(cerrdownloadpath,exist_ok=True)

In [17]:
xexp.resource('CERR').get(cerrdownloadpath,extract=True)

['demo\\UCLPTLU_02\\CERR\\UCLPTLU_02.mat.bz2']

### Get NIfTI files

In [18]:
xexp.resource('NIfTI').get(cerrdownloadpath,extract=True)

['demo\\UCLPTLU_02\\NIfTI\\CT_scan_1_CT.1062013.111435.370752.677178.nii',
 'demo\\UCLPTLU_02\\NIfTI\\PT_1_mask_GTV_CT.1062013.111627.3447983.0525999.nii',
 'demo\\UCLPTLU_02\\NIfTI\\PT_1_scan_2_CT.1062013.111627.3447983.0525128.nii',
 'demo\\UCLPTLU_02\\NIfTI\\PT_2_mask_GTV_CT.1062013.112120.4427669.175332.nii',
 'demo\\UCLPTLU_02\\NIfTI\\PT_2_scan_3_CT.1062013.112120.4427669.1753171.nii']

### Get DICOM files

In [17]:
scanlist = xexp.scans('*').get()
print(scanlist)

['1_3_6_1_4_1_9590_100_1_2_251798578225648820027105844481971439166', '1_3_6_1_4_1_9590_100_1_2_350146998316667394530049581082729940716', '1_3_6_1_4_1_9590_100_1_2_388534113514407529223602170040924316478', '1_3_6_1_4_1_9590_100_1_2_429175419830198836429998868710063379239']


In [19]:
for scan_id in scanlist:
    xscan = xexp.scan(scan_id)
    dicompath = os.path.join(download_dir,label_id,scan_id)    
    if xscan.resource('DICOM').exists():
        os.makedirs(dicompath,exist_ok=True)
        print(scan_id)
        xscan.resource('DICOM').get(dicompath)

1_3_6_1_4_1_9590_100_1_2_251798578225648820027105844481971439166
1_3_6_1_4_1_9590_100_1_2_350146998316667394530049581082729940716
1_3_6_1_4_1_9590_100_1_2_429175419830198836429998868710063379239
